In [14]:
import tensorflow as tf
import pandas as pd
import os
import requests
import json
import base64

In [50]:
df = pd.read_csv(os.path.join('data', 'new_spam.csv'))['email']
pred = df.sample(1)
pred

21    Get ready for our flash sale! Limited-time off...
Name: email, dtype: object

In [51]:
def prepare_json(text):
    feature_spec = {
        "email": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

In [52]:
def make_predictions(text):
    json_data = prepare_json(text)
    
    endpoint = "http://localhost:10001/v1/models/spam-detection-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    return "Berita Faktual" if prediction > 0.9 else "Berita Palsu"

In [53]:
make_predictions(pred.values[0])

'Berita Palsu'